In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
import os
import cv2
import cupy as cp
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np

In [ ]:
output_dir = '/content/drive/MyDrive/Video13each'

In [ ]:
# Load VGG19 model
vgg_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

80134624/80134624 [==============================] - 1s 0us/step


In [ ]:
def gpu_preprocess_frame(frame):
    frame = cp.asnumpy(frame)  # Convert CuPy array to numpy array
    frame = cv2.resize(frame, (224, 224))
    frame = preprocess_input(frame)
    return frame

def gpu_extract_features(frames):
    features = []
    for frame in frames:
        frame = gpu_preprocess_frame(frame)
        feature = tf.expand_dims(frame, axis=0)
        feature = vgg_model.predict(feature)
        features.append(feature)
    return cp.array(features)

def gpu_read_frames(video_file):
    cap = cv2.VideoCapture(video_file)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(cp.asarray(frame))
    cap.release()
    return frames

In [ ]:
# Step 1: Load videos and extract features
X = []
y = []

# Iterate over the folders in the directory containing the videos
for label in os.listdir(output_dir):
    label_dir = os.path.join(output_dir, label)
    # Check if the item in the directory is a folder
    if os.path.isdir(label_dir):
        # Iterate through the videos in the folder
        for video_file in os.listdir(label_dir):
            # Construct the full path to the video file
            video_path = os.path.join(label_dir, video_file)
            frames = gpu_read_frames(video_path)
            if len(frames) > 0:
                features = gpu_extract_features(frames)
                X.append(features)
                y.append(label)

1/1 [==============================] - 0s 20ms/step


In [ ]:
X = [x.get() for x in X]

In [ ]:
max_frames = max(features.shape[0] for features in X)
print(max_frames)

115


In [ ]:
for i in range(len(X)):
    # Calculate the amount of padding needed for each dimension
    pad_width = [(0, max_frames - X[i].shape[0])]  # Pad frames
    pad_width += [(0, 0)] * (X[i].ndim - 1)  # Pad other dimensions with zeros

    # Pad the array
    X[i] = np.pad(X[i], pad_width, mode='constant', constant_values=0)

In [ ]:
np.save('X_before_processing1.npy', X)

In [ ]:
X = np.array(X)

In [ ]:
y = np.array(y)

In [ ]:
np.save('X_video_features.npy', X)

In [ ]:
np.save('y_video_labels.npy', y)

In [ ]:
X = np.load('X_video_features.npy')

In [ ]:
y = np.load('y_video_labels.npy')

In [ ]:
import gc

# Perform memory cleanup
gc.collect()

172

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
# Split the dataset into training and testing sets, ensuring stratification
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:
np.save('X_train_vid.npy', X_train)
np.save('X_test_vid.npy', X_test)
np.save('y_train_vid.npy', y_train)
np.save('y_test_vid.npy', y_test)

In [ ]:
X_train = np.load('X_train_vid.npy')
X_test = np.load('X_test_vid.npy')


In [ ]:
y_train = np.load('y_train_vid.npy')
y_test = np.load('y_test_vid.npy')

In [ ]:
X_train_reshaped = np.array(X_train).reshape(-1, 115, 7, 7, 512)
X_test_reshaped = np.array(X_test).reshape(-1, 115, 7, 7, 512)

In [ ]:
np.save('X_train_reshaped.npy', X_train_reshaped)
np.save('X_test_reshaped.npy', X_test_reshaped)

In [ ]:
X_train_reshaped = np.load('X_train_reshaped.npy')
X_test_reshaped = np.load('X_test_reshaped.npy')

In [ ]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Dropout
from tensorflow.keras.optimizers import Adamr
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Build the model
model = Sequential([
    TimeDistributed(Flatten(), input_shape=(None, 7, 7, 512)),  # Flatten each time step's feature map
    Bidirectional(LSTM(256, return_sequences=True)),
    LSTM(128),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(set(y_encoded)), activation='softmax')
])

In [ ]:
# Step 7: Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Step 8: Define callbacks
checkpoint = ModelCheckpoint('sign_video_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

In [ ]:
# Step 9: Train the model
history = model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_test_reshaped, y_test), callbacks=[checkpoint])

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.6864 - accuracy: 0.7500
Epoch 1: val_accuracy improved from -inf to 0.00000, saving model to sign_video_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 9s 9s/step - loss: 0.6864 - accuracy: 0.7500 - val_loss: 0.7969 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.6576 - accuracy: 0.8750
Epoch 2: val_accuracy did not improve from 0.00000
1/1 [==============================] - 0s 119ms/step - loss: 0.6576 - accuracy: 0.8750 - val_loss: 0.8647 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.6351 - accuracy: 0.5000
Epoch 3: val_accuracy did not improve from 0.00000
1/1 [==============================] - 0s 114ms/step - loss: 0.6351 - accuracy: 0.5000 - val_loss: 0.9599 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.6095 - accuracy: 0.7500
Epoch 4: val_accuracy did not improve from 0.00000
1/1 [==============================] - 0s 113ms/step - loss: 0.6095 - accuracy: 0.7500 - val_loss: 1.0123 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==========================

In [ ]:
history.save('finalsign_video_model.h5')

In [ ]:
# Step 10: Evaluate the model
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')

1/1 [==============================] - 0s 46ms/step - loss: 1.3349 - accuracy: 0.5000
Validation Loss: 1.3349109888076782, Validation Accuracy: 0.5
